In [ ]:
import os
import glob as glob
import tqdm
import shutil
import math
import requests
import random
import functools
import logging
import json
from timeit import default_timer as timer

from multiprocessing import Process
from queue import Queue
from threading import Thread

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
import cv2

from torch.utils.data import *
from torch.utils.data.sampler import *

import torchvision
import torch

import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parallel.data_parallel import data_parallel

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import albumentations as albu
import albumentations.pytorch as AT

In [ ]:
threshold_label      = [ 0.75, 0.75, 0.5, 0.5]
threshold_mask_pixel = [ 0.5, 0.5, 0.5, 0.5]
threshold_mask_size  = [1,1,1,1]

In [ ]:
DATA_DIR    = '/home/avinash/Desktop/DL/severstal-steel-defect-detection/data'
SUBMISSION_CSV_FILE = 'submission.csv'

In [ ]:
IS_PYTORCH_PAD = True
IS_GATHER_EXCITE = False
def drop_connect(x, probability, training):
    if not training: return x

    batch_size = len(x)
    keep_probability = 1 - probability
    noise = keep_probability
    noise += torch.rand([batch_size, 1, 1, 1], dtype=x.dtype, device=x.device)
    mask = torch.floor(noise)
    x = x / keep_probability * mask

    return x
def upsize_add(x, lateral):
    return F.interpolate(x, size=lateral.shape[2:], mode='nearest') + lateral

def upsize(x, scale_factor=2):
    x = F.interpolate(x, scale_factor=scale_factor, mode='nearest')
    return x

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class Identity(nn.Module):
    def forward(self, x):
        return x

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(len(x),-1)

class Conv2dBn(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size=1, stride=1, zero_pad=[0,0,0,0], group=1):
        super(Conv2dBn, self).__init__()
        if IS_PYTORCH_PAD: zero_pad = [kernel_size//2]*4
        self.pad  = nn.ZeroPad2d(zero_pad)
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, padding=0, stride=stride, groups=group, bias=False)
        self.bn   = nn.BatchNorm2d(out_channel, eps=1e-03, momentum=0.01)

    def forward(self, x):
        x = self.pad (x)
        x = self.conv(x)
        x = self.bn  (x)
        return x

class SqueezeExcite(nn.Module):
    def __init__(self, in_channel, reduction_channel, excite_size):
        super(SqueezeExcite, self).__init__()
        self.excite_size=excite_size

        self.squeeze = nn.Conv2d(in_channel, reduction_channel, kernel_size=1, padding=0)
        self.excite  = nn.Conv2d(reduction_channel, in_channel, kernel_size=1, padding=0)
        self.act = Swish()

    def forward(self, x):

        if IS_GATHER_EXCITE:
            s = F.avg_pool2d(x, kernel_size=self.excite_size)
        else:
            s = F.adaptive_avg_pool2d(x,1)

        s = self.act(self.squeeze(s))
        s = torch.sigmoid(self.excite(s))

        if IS_GATHER_EXCITE:
            s = F.interpolate(s, size=(x.shape[2],x.shape[3]), mode='nearest')

        x = s*x
        return x

class EfficientBlock(nn.Module):

    def __init__(self, in_channel, channel, out_channel, kernel_size, stride, zero_pad, excite_size, drop_connect_rate):
        super().__init__()
        self.is_shortcut = stride == 1 and in_channel == out_channel
        self.drop_connect_rate = drop_connect_rate

        if in_channel == channel:
            self.bottleneck = nn.Sequential(
                Conv2dBn(   channel, channel, kernel_size=kernel_size, stride=stride, zero_pad=zero_pad, group=channel),
                Swish(),
                SqueezeExcite(channel, in_channel//4, excite_size) if excite_size>0
                else Identity(),
                Conv2dBn(channel, out_channel, kernel_size=1, stride=1),
            )
        else:
            self.bottleneck = nn.Sequential(
                Conv2dBn(in_channel, channel, kernel_size=1, stride=1),
                Swish(),
                Conv2dBn(   channel, channel, kernel_size=kernel_size, stride=stride, zero_pad=zero_pad, group=channel),
                Swish(),
                SqueezeExcite(channel, in_channel//4, excite_size) if excite_size>0
                else Identity(),
                Conv2dBn(channel, out_channel, kernel_size=1, stride=1)
            )

    def forward(self, x):
        b = self.bottleneck(x)

        if self.is_shortcut:
            if self.training: b = drop_connect(b, self.drop_connect_rate, True)
            x = b + x
        else:
            x = b
        return x

class ConvGnUp2d(nn.Module):
    def __init__(self, in_channel, out_channel, num_group=32, kernel_size=3, padding=1, stride=1):
        super(ConvGnUp2d, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, padding=padding, stride=stride, bias=False)
        self.gn   = nn.GroupNorm(num_group,out_channel)

    def forward(self,x):
        x = self.conv(x)
        x = self.gn(x)
        x = F.relu(x, inplace=True)
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
        return x
    

class EfficientNetB5(nn.Module):

    def __init__(self, drop_connect_rate=0.4):
        super(EfficientNetB5, self).__init__()
        d = drop_connect_rate

        # bottom-top
        self.stem  = nn.Sequential(
            Conv2dBn(3,48, kernel_size=3,stride=2,zero_pad=[0,1,0,1]),
            Swish()
        )

        self.block1 = nn.Sequential(
               EfficientBlock( 48,  48,  24, kernel_size=3, stride=1, zero_pad=[1,1,1,1], excite_size=128, drop_connect_rate=d*1/7),
            * [EfficientBlock( 24,  24,  24, kernel_size=3, stride=1, zero_pad=[1,1,1,1], excite_size=128, drop_connect_rate=d*1/7) for i in range(1,3)],
        )
        self.block2 = nn.Sequential(
               EfficientBlock( 24, 144,  40, kernel_size=3, stride=2, zero_pad=[0,1,0,1], excite_size= 64, drop_connect_rate=d*2/7),
            * [EfficientBlock( 40, 240,  40, kernel_size=3, stride=1, zero_pad=[1,1,1,1], excite_size= 64, drop_connect_rate=d*2/7) for i in range(1,5)],
        )
        self.block3 = nn.Sequential(
               EfficientBlock( 40, 240,  64, kernel_size=5, stride=2, zero_pad=[1,2,1,2], excite_size= 32, drop_connect_rate=d*3/7),
            * [EfficientBlock( 64, 384,  64, kernel_size=5, stride=1, zero_pad=[2,2,2,2], excite_size= 32, drop_connect_rate=d*3/7) for i in range(1,5)],
        )
        self.block4 = nn.Sequential(
               EfficientBlock( 64, 384, 128, kernel_size=3, stride=2, zero_pad=[0,1,0,1], excite_size= 16, drop_connect_rate=d*4/7),
            * [EfficientBlock(128, 768, 128, kernel_size=3, stride=1, zero_pad=[1,1,1,1], excite_size= 16, drop_connect_rate=d*4/7) for i in range(1,7)],
        )
        self.block5 = nn.Sequential(
               EfficientBlock(128, 768, 176, kernel_size=5, stride=1, zero_pad=[2,2,2,2], excite_size= 16, drop_connect_rate=d*5/7),
            * [EfficientBlock(176,1056, 176, kernel_size=5, stride=1, zero_pad=[2,2,2,2], excite_size= 16, drop_connect_rate=d*5/7) for i in range(1,7)],
        )
        self.block6 = nn.Sequential(
               EfficientBlock(176,1056, 304, kernel_size=5, stride=2, zero_pad=[1,2,1,2], excite_size=  8, drop_connect_rate=d*6/7),
            * [EfficientBlock(304,1824, 304, kernel_size=5, stride=1, zero_pad=[2,2,2,2], excite_size=  8, drop_connect_rate=d*6/7) for i in range(1,9)],
        )
        self.block7 = nn.Sequential(
               EfficientBlock(304,1824, 512, kernel_size=3, stride=1, zero_pad=[1,1,1,1], excite_size=  8, drop_connect_rate=d*7/7),
            * [EfficientBlock(512,3072, 512, kernel_size=3, stride=1, zero_pad=[1,1,1,1], excite_size=  8, drop_connect_rate=d*7/7) for i in range(1,3)],
        )

        self.last = nn.Sequential(
            Conv2dBn(512, 2048,kernel_size=1,stride=1),
            Swish()
        )

        self.logit = nn.Linear(2048,1000)

    def forward(self, x):
        batch_size = len(x)

        x = self.stem(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)
        x = self.last(x)

        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        logit = self.logit(x)

        return logit

class EfficientNetB5Segmentation(nn.Module):

    def __init__(self, num_class=4, drop_connect_rate=0.2):
        super(EfficientNetB5Segmentation, self).__init__()

        e = EfficientNetB5(drop_connect_rate)
        self.stem   = e.stem
        self.block1 = e.block1
        self.block2 = e.block2
        self.block3 = e.block3
        self.block4 = e.block4
        self.block5 = e.block5
        self.block6 = e.block6
        self.block7 = e.block7
        self.last   = e.last
        e = None  #dropped

        #---
        self.lateral0 = nn.Conv2d(2048, 64,  kernel_size=1, padding=0, stride=1)
        self.lateral1 = nn.Conv2d( 176, 64,  kernel_size=1, padding=0, stride=1)
        self.lateral2 = nn.Conv2d(  64, 64,  kernel_size=1, padding=0, stride=1)
        self.lateral3 = nn.Conv2d(  40, 64,  kernel_size=1, padding=0, stride=1)

        self.top1 = nn.Sequential(
            ConvGnUp2d( 64, 64),
            ConvGnUp2d( 64, 64),
            ConvGnUp2d( 64, 64),
        )
        self.top2 = nn.Sequential(
            ConvGnUp2d( 64, 64),
            ConvGnUp2d( 64, 64),
        )
        self.top3 = nn.Sequential(
            ConvGnUp2d( 64, 64),
        )
        self.top4 = nn.Sequential(
            nn.Conv2d(64*3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.logit_mask = nn.Conv2d(64,1+num_class,kernel_size=1)

    def forward(self, x):
        batch_size,C,H,W = x.shape

        x = self.stem(x)            #; print('stem  ',x.shape)
        x = self.block1(x)    ;x0=x #; print('block1',x.shape)
        x = self.block2(x)    ;x1=x #; print('block2',x.shape)
        x = self.block3(x)    ;x2=x #; print('block3',x.shape)
        x = self.block4(x)          #; print('block4',x.shape)
        x = self.block5(x)    ;x3=x #; print('block5',x.shape)
        x = self.block6(x)          #; print('block6',x.shape)
        x = self.block7(x)          #; print('block7',x.shape)
        x = self.last(x)      ;x4=x #; print('last  ',x.shape)

        # segment
        t0 = self.lateral0(x4)
        t1 = upsize_add(t0, self.lateral1(x3)) #16x16
        t2 = upsize_add(t1, self.lateral2(x2)) #32x32
        t3 = upsize_add(t2, self.lateral3(x1)) #64x64

        t1 = self.top1(t1) #128x128
        t2 = self.top2(t2) #128x128
        t3 = self.top3(t3) #128x128

        t = torch.cat([t1,t2,t3],1)
        t = self.top4(t)
        logit_mask = self.logit_mask(t)
        logit_mask = F.interpolate(logit_mask, scale_factor=2.0, mode='bilinear', align_corners=False)

        return logit_mask

In [ ]:
def probability_mask_to_probability_label(probability):
    batch_size,num_class,H,W = probability.shape
    probability = probability.permute(0, 2, 3, 1).contiguous().view(batch_size,-1, 5)
    value, index = probability.max(1)
    probability = value
    return probability

def image_to_input(image):
    input = image.astype(np.float32)
    input = input[...,::-1]/255
    input = input.transpose(0,3,1,2)
    # input[:,0] = (input[:,0]-IMAGE_RGB_MEAN[0])/IMAGE_RGB_STD[0]
    # input[:,1] = (input[:,1]-IMAGE_RGB_MEAN[1])/IMAGE_RGB_STD[1]
    # input[:,2] = (input[:,2]-IMAGE_RGB_MEAN[2])/IMAGE_RGB_STD[2]
    return input


class KaggleTestDataset(Dataset):
    def __init__(self):

        df =  pd.read_csv(DATA_DIR + '/sample_submission.csv')
        df['ImageId'] = df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
        self.uid = df['ImageId'].unique().tolist()

    def __str__(self):
        string  = ''
        string += '\tlen = %d\n'%len(self)
        return string

    def __len__(self):
        return len(self.uid)

    def __getitem__(self, index):
        # print(index)
        image_id = self.uid[index]
        image = cv2.imread(DATA_DIR + '/test_images/%s'%(image_id), cv2.IMREAD_COLOR)
        return image, image_id


def null_collate(batch):
    batch_size = len(batch)
    input = []
    image_id = []
    for b in range(batch_size):
        input.append(batch[b][0])
        image_id.append(batch[b][1])
    input = np.stack(input)
    input = torch.from_numpy(image_to_input(input))
    return input, image_id



### kaggle ##############################################################

def post_process(mask, min_size):
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))

    predict = np.zeros((256, 1600), np.float32)
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predict[p] = 1

    return predict


def run_length_encode(mask):
    #https://www.kaggle.com/bigkizd/se-resnext50-89
    m = mask.T.flatten()
    if m.sum()==0:
        rle=''
    else:
        m   = np.concatenate([[0], m, [0]])
        run = np.where(m[1:] != m[:-1])[0] + 1
        run[1::2] -= run[::2]
        rle = ' '.join(str(r) for r in run)
    return rle

In [ ]:
#########################################################################

def run_check_setup():
    
    ## load net
    net = EfficientNetB5Segmentation().cuda()
    net.load_state_dict(torch.load(CHECKPOINT_FILE, map_location=lambda storage, loc: storage),strict=True)

    ## load data
    image_id = ['004f40c73.jpg', '006f39c41.jpg', '00b7fb703.jpg', '00bbcd9af.jpg']
    image=[]
    for i in image_id:
        m = cv2.imread(DATA_DIR +'/test_images/%s'%i)
        image.append(m)
    image = np.stack(image)
    input = image_to_input(image)
    input = torch.from_numpy(input).cuda()

    #run here!
    net.eval()
    with torch.no_grad():
        logit = net(input)
        probability= torch.sigmoid(logit)

    print('input: ',input.shape)
    print('logit: ',logit.shape)
    print('')
    #---
    input = input.data.cpu().numpy()
    logit = logit.data.cpu().numpy()

    if 1:
        print(logit[0,0,:5,:5],'\n')
        print(logit[3,0,-5:,-5:],'\n')
        print(logit.mean(),logit.std(),logit.max(),logit.min(),'\n')
        print('')
        print('---------------------')
        print('')


'''
        
input:  torch.Size([4, 3, 256, 1600])
logit:  torch.Size([4, 5, 256, 1600])

[[5.097667  5.279234  5.642368  5.863943  5.9439597]
 [5.122505  5.325696  5.7320786 5.976972  6.060377 ]
 [5.172181  5.4186206 5.9115    6.2030306 6.293213 ]
 [5.2060194 5.475979  6.0158978 6.331575  6.42301  ]
 [5.2240195 5.4977703 6.0452724 6.362605  6.449769 ]] 

[[5.0015    4.846141  4.578332  4.198073  4.0079427]
 [4.9045486 4.746123  4.4800625 4.1063666 3.9195185]
 [4.7522364 4.5993853 4.368871  4.0606937 3.9066048]
 [4.544562  4.4059286 4.244759  4.061054  3.9692013]
 [4.440725  4.3092003 4.182703  4.0612345 4.0004997]] 

0.035303652 3.9407735 10.5579815 -6.170837 

        
'''

In [ ]:
def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)

    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)

    else:
        raise NotImplementedError

In [ ]:
def summarise_submission_csv(df):


    text = ''
    df['Class'] = df['ImageId_ClassId'].str[-1].astype(np.int32)
    df['Label'] = (df['EncodedPixels']!='').astype(np.int32)
    num_image = len(df)//4
    num = len(df)

    pos = (df['Label']==1).sum()
    neg = num-pos


    pos1 = ((df['Class']==1) & (df['Label']==1)).sum()
    pos2 = ((df['Class']==2) & (df['Label']==1)).sum()
    pos3 = ((df['Class']==3) & (df['Label']==1)).sum()
    pos4 = ((df['Class']==4) & (df['Label']==1)).sum()

    neg1 = num_image-pos1
    neg2 = num_image-pos2
    neg3 = num_image-pos3
    neg4 = num_image-pos4


    text += 'compare with LB probing ... \n'
    text += '\t\tnum_image = %5d(1801) \n'%num_image
    text += '\t\tnum  = %5d(7204) \n'%num
    text += '\n'

    text += '\t\tpos1 = %5d( 128)  %0.3f\n'%(pos1,pos1/128)
    text += '\t\tpos2 = %5d(  43)  %0.3f\n'%(pos2,pos2/43)
    text += '\t\tpos3 = %5d( 741)  %0.3f\n'%(pos3,pos3/741)
    text += '\t\tpos4 = %5d( 120)  %0.3f\n'%(pos4,pos4/120)
    text += '\n'

    text += '\t\tneg1 = %5d(1673)  %0.3f  %3d\n'%(neg1,neg1/1673, neg1-1673)
    text += '\t\tneg2 = %5d(1758)  %0.3f  %3d\n'%(neg2,neg2/1758, neg2-1758)
    text += '\t\tneg3 = %5d(1060)  %0.3f  %3d\n'%(neg3,neg3/1060, neg3-1060)
    text += '\t\tneg4 = %5d(1681)  %0.3f  %3d\n'%(neg4,neg4/1681, neg4-1681)
    text += '--------------------------------------------------\n'
    text += '\t\tneg  = %5d(6172)  %0.3f  %3d \n'%(neg,neg/6172, neg-6172)
    text += '\n'

    if 1:
        #compare with reference
        pass

    return text


In [ ]:
########################################################################

def run_make_submission_csv():

#     threshold_label      = [ 0.50, 0.50, 0.50, 0.50,]
#     threshold_mask_pixel = [ 0.50, 0.50, 0.50, 0.50,]
#     threshold_mask_size  = [ 1,  1,  1,  1,]
    threshold_label = [0.75, 0.75, 0.5, 0.5]
    threshold_mask_pixel = [0.5, 0.5, 0.5, 0.5]
    threshold_mask_size  = [900, 900, 900, 2000]
    ## load net
    print('load net ...')
#     lt = [40000, 36000, 45000, 34000, 42000, 41000]
    
    for i,file in enumerate(glob.glob('/home/avinash/efficientb5-fpn-crop256x400-foldb0c/checkpoint/*_model.pth')):
        
        val = file.split('/')[-1]
        val = val.split('_')[0]
        print(val)
#         if int(val) in lt:
        SUBMISSION_CSV_FILE = '/home/avinash/efficientb5-fpn-crop256x400-foldb0c/submit/submission'+val+'.csv'
        net = EfficientNetB5Segmentation().cuda()
        net.load_state_dict(torch.load(file, map_location=lambda storage, loc: storage))
        print('')


        ## load data
        print('load data ...')
        dataset = KaggleTestDataset()
        print(dataset)
        #exit(0)

        loader  = DataLoader(
            dataset,
            sampler     = SequentialSampler(dataset),
            batch_size  = 8,
            drop_last   = False,
            num_workers = 0,
            pin_memory  = True,
            collate_fn  = null_collate
        )


        ## start here ----------------------------------
        image_id_class_id = []
        encoded_pixel     = []


        net.eval()

        start_timer = timer()
        for t,(input, image_id) in enumerate(loader):
            if t%50==0:
                print('\r loader: t = %4d / %4d  %s  %s : %s'%(
                      t, len(loader)-1, str(input.shape), image_id[0], time_to_str((timer() - start_timer),'sec'),
                ),end='', flush=True)

            with torch.no_grad():
                input = input.cuda()

                logit = data_parallel(net,input)  #net(input)
                probability = torch.softmax(logit,1)

                probability_mask  = probability[:,1:]#just drop background
                probability_label = probability_mask_to_probability_label(probability)[:,1:]


            probability_mask  = probability_mask.data.cpu().numpy()
            probability_label = probability_label.data.cpu().numpy()

            batch_size = len(image_id)
            for b in range(batch_size):
                for c in range(4):
                    rle=''

                    predict_label = probability_label[b,c]>threshold_label[c]
                    if predict_label:
                        try:
                            predict_mask = probability_mask[b,c] > threshold_mask_pixel[c]
                            predict_mask = post_process(predict_mask, threshold_mask_size[c])
                            rle = run_length_encode(predict_mask)

                        except:
                            print('An exception occurred : %s'%(image_id[b]+'_%d'%(c+1)))


                    image_id_class_id.append(image_id[b]+'_%d'%(c+1))
                    encoded_pixel.append(rle)


        df = pd.DataFrame(zip(image_id_class_id, encoded_pixel), columns=['ImageId_ClassId', 'EncodedPixels'])
        df.to_csv(SUBMISSION_CSV_FILE, index=False)
        print('')

        ## print statistics ----
        if 1:
            text = summarise_submission_csv(df)
            print(text)



# main #################################################################
if __name__ == '__main__':
    #run_check_setup()
    run_make_submission_csv()

    print('\nsucess!')

In [ ]:
# # Dataset setup
# class TestDataset(Dataset):
#     '''Dataset for test prediction'''
#     def __init__(self, root, df, mean, std):
#         self.root = root
#         df['ImageId'] = df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
#         self.fnames = df['ImageId'].unique().tolist()
#         self.num_samples = len(self.fnames)
#         self.transform = albu.Compose(
#             [
#                 albu.Normalize(),
#                 AT.ToTensor()
#             ]
#         )

#     def __getitem__(self, idx):
#         fname = self.fnames[idx]
#         path = os.path.join(self.root, fname)
#         image = cv2.imread(path)
#         images = self.transform(image=image)["image"]
#         return fname, images

#     def __len__(self):
#         return self.num_samples
    
# sample_submission_path = '../input/severstal-steel-defect-detection/sample_submission.csv'
# test_data_folder = "../input/severstal-steel-defect-detection/test_images"

# # hyperparameters
# batch_size = 10

# # mean and std
# mean = (0.485, 0.456, 0.406)
# std = (0.229, 0.224, 0.225)

# df = pd.read_csv(sample_submission_path)

# # dataloader
# testset = DataLoader(
#     TestDataset(test_data_folder, df, mean, std),
#     batch_size=batch_size,
#     shuffle=False,
#     num_workers=0,
#     pin_memory=True
# )

In [ ]:
# model = EfficientNetB5Segmentation()

In [ ]:
# path = params['model_name']
# x = torch.load(path)
# model.load_state_dict(x)

In [ ]:
# model = model.cuda()
# def sharpen(p,t=0.5):
#     if t!=0:
#         return p**t
#     else:
#         return p

# test_num  = 0
# test_id   = []
# #test_image = []
# test_probability_label = [] # 8bit
# test_probability_mask  = [] # 8bit
# test_truth_label = []
# test_truth_mask  = []



# for image_id,images in tqdm.tqdm_notebook(testset):
#     images = images.cuda()
# #             truth_label = truth_label.cuda()
# #             truth_mask = truth_mask.cuda()
#     batch_size,C,H,W = images.shape
# #            print(images.shape)
# #            print(masks.shape)
#     with torch.no_grad():
#         model.eval()
#         num_augment = 0
#         if 1: #  null
#             logit =  data_parallel(model,images)  #net(input)
#             probability = torch.softmax(logit,1)

#             probability_mask = sharpen(probability,0)
#             num_augment+=1

# #                if 'flip_lr' in augment:
# #                    logit = data_parallel(net,torch.flip(input,dims=[3]))
# #                    probability  = torch.softmax(torch.flip(logit,dims=[3]),1)
# #    
# #                    probability_mask += sharpen(psrobability)
# #                    num_augment+=1
# #    
# #                if 'flip_ud' in augment:
# #                    logit = data_parallel(net,torch.flip(input,dims=[2]))
# #                    probability = torch.softmax(torch.flip(logit,dims=[2]),1)
# #    
# #                    probability_mask += sharpen(probability)
# #                    num_augment+=1
# #    
#         #---
#         probability_mask = probability_mask/num_augment
# #         probability_label = probability_mask_to_probability_label(probability_mask)

#     #---
# #             truth_label = truth_label.data.cpu().numpy().astype(np.uint8)
# #             truth_mask  = truth_mask.data.cpu().numpy().astype(np.uint8)
#     probability_mask = (probability_mask.data.cpu().numpy()*255).astype(np.uint8)
# #     probability_label = (probability_label.data.cpu().numpy()).astype(np.uint8)

#     test_probability_mask.extend(probability_mask)
# #     test_probability_label.extend(probability_label)
#     test_id.extend(image_id)
# #             test_truth_mask.append(truth_mask)
# #             test_truth_label.append(truth_label)
#     test_num += batch_size

In [ ]:
# test_id = np.array(test_id)
# test_probability_mask = np.array(test_probability_mask)
# test_probability_mask = np.concatenate(test_probability_mask)
# test_probability_label = np.concatenate(test_probability_label)
# test_id = np.concatenate(test_id)

In [ ]:
# test_probability_mask.shape

In [ ]:
#         print(test_id.shape)
#         print(test_probability_label.shape)
#         print(test_probability_mask.shape)
#         test_truth_mask  = np.concatenate(test_truth_mask)
#         test_truth_label = np.concatenate(test_truth_label)


#         value = np.max(test_probability_mask,1,keepdims=True)
#         value = test_probability_mask*(value==test_probability_mask)
# test_probability_mask = test_probability_mask[:,1:] #remove background class

#         index = np.ones((len(test_id),4,256,1600),np.uint8)*np.array([1,2,3,4],np.uint8).reshape(1,4,1,1)
#         truth_mask = truth_mask==index

In [ ]:
# res = []
# for p, file in zip(test_probability_mask, test_id):
#     for i in range(4):
#         p_channel = p[i]
#         imageid_classid = file+'_'+str(i+1)
#         p_channel = (p_channel>(threshold_mask_pixel[i]*255)).astype(np.uint8)
#         if p_channel.sum() < threshold_mask_size[i]:
#             p_channel = np.zeros(p_channel.shape, dtype=p_channel.dtype)

#         res.append({
#             'ImageId_ClassId': imageid_classid,
#             'EncodedPixels': run_length_encode(p_channel)
#         })

In [ ]:
# df = pd.DataFrame(res)
# df = df.fillna('')
# df.to_csv('submission.csv',index=False)

In [ ]:
# sum(df.EncodedPixels != '')